# ORIGINAL SOURCE CODE

Link in case you want to see from where it's from: https://discourse.bokeh.org/t/is-there-an-example-on-how-to-use-lod-on-patches/5757/2

Also note that the models can throw and error if you rerun them : Models must be owned by only a single document, GeoJSONDataSource(id='p1271', ...) is already in a doc

To fix, just restart the kernel and run the code again

In [2]:
import geopandas as gpd
from bokeh.io import show, output_file
from bokeh.plotting import figure
from bokeh.models import HoverTool
from bokeh.palettes import PRGn, RdYlGn
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter, BasicTicker

GeoData= gpd.read_file("test.geojson")
geosource = GeoJSONDataSource(geojson = GeoData.to_json())

TOOLTIPS = [
    ("Value", "@Range")
    ]
tools = 'wheel_zoom,pan,reset'

palette = PRGn[11]

color_mapper = LinearColorMapper(palette = palette, low = GeoData['Range'].min(), high = GeoData['Range'].max())

p = figure(title = "Test", width=800, 
           height=870, toolbar_location='right', tooltips=TOOLTIPS, tools=tools,
          x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(Vendors.OSM)
p.patches('xs','ys', source = geosource,fill_color = {'field' :'Range', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)


color_bar = ColorBar(color_mapper=color_mapper,  ticker= BasicTicker(),
formatter = NumeralTickFormatter(format='0.0[0000]'), 
label_standoff = 20, width=8, location=(0,0), padding = 5, title = "Price", major_tick_line_color = "black",major_tick_out = 8, major_label_text_font_size = "15px")
# Set color_bar location
p.add_layout(color_bar, 'right')

#p.lod_factor=8
#p.lod_interval=20
#p.lod_threshold=5
#p.lod_timeout=2

show(p)

In [9]:
GeoData.dtypes

Range          int64
geometry    geometry
dtype: object

In [11]:
chicago_df.dtypes

community                    object
NID                           int64
POP2010                       int64
POP2000                       int64
POPCH                         int64
POPPERCH                    float64
popplus                       int64
popneg                        int64
geometry                   geometry
geographic_cluster_name      object
km_to_bus_stop              float64
dtype: object

# CHANGE INTO MY CODE

In [3]:
import json
from bokeh.io import show
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider)
from bokeh.layouts import column, row
from bokeh.palettes import brewer
from bokeh.plotting import figure

from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure, ColumnDataSource
from bokeh.tile_providers import get_provider, Vendors
from bokeh.palettes import PRGn, RdYlGn
from bokeh.transform import linear_cmap,factor_cmap
from bokeh.layouts import row, column
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter, BasicTicker
import numpy as np
import pandas as pd
import geopandas as gpd
import geodatasets
from bokeh.io import show, output_notebook, output_file
from bokeh.models import (
    GeoJSONDataSource,
    HoverTool,
    LinearColorMapper
)
from bokeh.plotting import figure
from bokeh.palettes import Viridis6
from bokeh.models import ColumnDataSource, Grid, LinearAxis, MultiPolygons, Plot

In [5]:
data2 = pd.read_csv("mcmf.csv")
chicago = gpd.read_file(geodatasets.get_path("geoda.chicago_commpop"))
chicago['community'] = chicago['community'].apply(str.lower)
gdf = gpd.GeoDataFrame(data2, geometry=gpd.points_from_xy(data2.longitude, data2.latitude), crs="EPSG:4326")
gdf_grouped = gdf.groupby('geographic_cluster_name')['km_to_bus_stop'].mean().reset_index()
gdf_grouped = pd.DataFrame(gdf_grouped)
gdf_grouped['geographic_cluster_name'] = gdf_grouped['geographic_cluster_name'].apply(str.lower)
gdf_grouped['community'] = gdf_grouped['geographic_cluster_name']
chicago_df = pd.merge(chicago, gdf_grouped, on = 'community')
chicago_df = chicago_df.drop(columns = ["NID", "POP2010", "POP2000", "POPCH", "POPPERCH", "popplus", "popneg", "geographic_cluster_name"])
testing = chicago_df.explode()


# removed unncessary columns to see if that was a problem
# this testing dataframe was just to see if I needed to convert the MultiPolygon type column (geometry) into a Polygon type

/var/folders/3v/6br22y3x30q_cgj59k5b387w0000gn/T/ipykernel_46290/3730479445.py:1: DtypeWarning: Columns (14,28,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  data2 = pd.read_csv("mcmf.csv")
/var/folders/3v/6br22y3x30q_cgj59k5b387w0000gn/T/ipykernel_46290/3730479445.py:11: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  testing = chicago_df.explode()


In [7]:
geosource = GeoJSONDataSource(geojson = testing.to_json())

# converts dataframe into a json that can be used with bokeh

In [9]:
import geopandas as gpd
from bokeh.io import show, output_file
from bokeh.plotting import figure
from bokeh.models import HoverTool
from bokeh.palettes import PRGn, RdYlGn
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter, BasicTicker


TOOLTIPS = [
    ("Community", "@community"),
    ("Km to Bus Stop", "@km_to_bus_stop")
    ]
tools = 'wheel_zoom,pan,reset'

palette = PRGn[11]

color_mapper = LinearColorMapper(palette = palette, low = chicago_df['km_to_bus_stop'].min(), high = chicago_df['km_to_bus_stop'].max())

p = figure(title = "Test", width=800, 
           height=870, toolbar_location='right', tooltips=TOOLTIPS, tools=tools,
          x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(Vendors.OSM)

# This Vendors.OSM is the Open Street Map tile that basically puts the OSM map over the figure --> this is what's 
# not working on this code but works on the other one

p.patches('xs','ys', source = geosource,fill_color = {'field' :'km_to_bus_stop', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
# the p.patches is how you include the chicago clusters into the figure



color_bar = ColorBar(color_mapper=color_mapper,  ticker= BasicTicker(),
formatter = NumeralTickFormatter(format='0.0[0000]'), 
label_standoff = 20, width=8, location=(0,0), padding = 5, title = "Price", major_tick_line_color = "black",major_tick_out = 8, major_label_text_font_size = "15px")
# Set color_bar location
p.add_layout(color_bar, 'right')

#p.lod_factor=8
#p.lod_interval=20
#p.lod_threshold=5
#p.lod_timeout=2

show(p)

RuntimeError: Models must be owned by only a single document, GeoJSONDataSource(id='p1271', ...) is already in a doc